### DB 연결

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
import time
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine

In [2]:
#pip freeze > requirements.txt

In [3]:
import psycopg2
from sshtunnel import SSHTunnelForwarder


try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


### 테이블확인

In [ ]:


d_labitems
microbiologyevents
prescriptions

In [4]:
curs.execute("select * from mimiciv_hosp.d_labitems;")
columns_name = [desc[0] for desc in curs.description]
d_labitems = pd.DataFrame(curs.fetchall(), columns=columns_name) 
d_labitems

,itemid,label,fluid,category
0,50801,Alveolar-arterial Gradient,Blood,Blood Gas
1,50802,Base Excess,Blood,Blood Gas
2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas
3,50804,Calculated Total CO2,Blood,Blood Gas
4,50805,Carboxyhemoglobin,Blood,Blood Gas
...,...,...,...,...
1617,53150,Anti Hbs,Blood,Chemistry
1618,53151,Anti-la,Blood,Chemistry
1619,53152,HIV FINAL,Blood,Chemistry
1620,53153,HIV Screen,Blood,Chemistry


In [11]:
#labitems 에서 label 에 paco2가 있는지 확인
bloodgas_list = d_labitems >> mask(X.category == 'Blood Gas')
bloodgas_list

,itemid,label,fluid,category
0,50801,Alveolar-arterial Gradient,Blood,Blood Gas
1,50802,Base Excess,Blood,Blood Gas
2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas
3,50804,Calculated Total CO2,Blood,Blood Gas
4,50805,Carboxyhemoglobin,Blood,Blood Gas
...,...,...,...,...
1140,52047,"Sodium, Urine",Urine,Blood Gas
1518,52434,"Chloride, Whole Blood",Blood,Blood Gas
1519,52442,Lactate,Blood,Blood Gas
1520,52452,"Potassium, Whole Blood",Blood,Blood Gas


In [18]:
bloodgas_list[bloodgas_list['label'].isin(['pCO2','pH','pO2'])]
#고쳐줘


,itemid,label,fluid,category
16,50818,pCO2,Blood,Blood Gas
18,50820,pH,Blood,Blood Gas
19,50821,pO2,Blood,Blood Gas
29,50831,pH,Other Body Fluid,Blood Gas
1133,52040,pCO2,Fluid,Blood Gas
1134,52041,pH,Fluid,Blood Gas
1135,52042,pO2,Fluid,Blood Gas


In [12]:
#bloodgas_list.to_csv('bloodgas_list.txt')
#label pCO2 pH PO2 

In [17]:
curs.execute("select * from mimiciv_hosp.labevents where itemid in (50818,50820);")
columns_name = [desc[0] for desc in curs.description]
lab_abga = pd.DataFrame(curs.fetchall(), columns=columns_name) 
lab_abga

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,17891825,11538348,22202995.0,1375778,50818,None,2127-06-25 09:09:00,2127-06-25 09:11:00,41,41.00,mm Hg,35.00,45.00,None,None,None
1,17891826,11538348,22202995.0,1375778,50820,None,2127-06-25 09:09:00,2127-06-25 09:11:00,7.42,7.42,units,7.35,7.45,None,None,None
2,17891840,11538348,22202995.0,7878587,50818,None,2127-06-25 13:13:00,2127-06-25 13:15:00,40,40.00,mm Hg,35.00,45.00,None,None,None
3,17891841,11538348,22202995.0,7878587,50820,None,2127-06-25 13:13:00,2127-06-25 13:15:00,7.38,7.38,units,7.35,7.45,None,None,None
4,17891855,11538348,22202995.0,42940959,50818,None,2127-06-25 13:55:00,2127-06-25 13:59:00,41,41.00,mm Hg,35.00,45.00,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1076120,17883430,11538029,26960640.0,81720925,50820,None,2133-02-09 12:16:00,2133-02-09 12:18:00,7.39,7.39,units,7.35,7.45,None,None,None
1076121,17884997,11538053,29073118.0,20807031,50818,None,2186-11-27 15:00:00,2186-11-27 15:03:00,27,27.00,mm Hg,35.00,45.00,abnormal,None,None
1076122,17884998,11538053,29073118.0,20807031,50820,None,2186-11-27 15:00:00,2186-11-27 15:03:00,7.43,7.43,units,7.35,7.45,None,None,None
1076123,17886621,11538083,NaN,8663830,50818,None,2188-11-16 09:35:00,2188-11-16 09:36:00,43,43.00,mm Hg,35.00,45.00,None,None,None


In [19]:
lab_abga.to_csv('../outputs/lab_abga.csv', index=False, encoding='utf-8-sig')
lab_abga = pd.read_csv('../outputs/lab_abga.csv', parse_dates=['charttime','storetime'])

In [20]:
lab_abga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076125 entries, 0 to 1076124
Data columns (total 16 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   labevent_id        1076125 non-null  int64         
 1   subject_id         1076125 non-null  int64         
 2   hadm_id            966571 non-null   float64       
 3   specimen_id        1076125 non-null  int64         
 4   itemid             1076125 non-null  int64         
 5   order_provider_id  5604 non-null     object        
 6   charttime          1076125 non-null  datetime64[ns]
 7   storetime          1076109 non-null  datetime64[ns]
 8   value              1075496 non-null  object        
 9   valuenum           1075496 non-null  float64       
 10  valueuom           1076125 non-null  object        
 11  ref_range_lower    1076125 non-null  float64       
 12  ref_range_upper    1076125 non-null  float64       
 13  flag               531149 n

In [21]:
subjectlist1 = pd.read_csv('./data/subjectlist1.csv', parse_dates=['admittime','intubationtime','dischtime', 'deathtime','extubationtime','reintubation_eventtime'])
subjectlist1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9003 entries, 0 to 9002
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              9003 non-null   int64         
 1   subject_id              9003 non-null   int64         
 2   hadm_id                 9003 non-null   int64         
 3   stay_id                 9003 non-null   int64         
 4   int_stayid              7860 non-null   float64       
 5   admittime               9003 non-null   datetime64[ns]
 6   intubationtime          7860 non-null   datetime64[ns]
 7   int_itemid              7860 non-null   float64       
 8   int_weight              7860 non-null   float64       
 9   ext_stayid              6825 non-null   float64       
 10  extubationtime          8759 non-null   datetime64[ns]
 11  ext_itemid              6825 non-null   float64       
 12  ext_weight              6825 non-null   float64 

In [22]:
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id')

In [24]:
sub1_abga = sub1list >> left_join(lab_abga, by=['subject_id','hadm_id'])

In [25]:
sub1_abga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464515 entries, 0 to 464514
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   subject_id         464515 non-null  int64         
 1   hadm_id            464515 non-null  int64         
 2   stay_id            464515 non-null  int64         
 3   labevent_id        464163 non-null  float64       
 4   specimen_id        464163 non-null  float64       
 5   itemid             464163 non-null  float64       
 6   order_provider_id  6 non-null       object        
 7   charttime          464163 non-null  datetime64[ns]
 8   storetime          464150 non-null  datetime64[ns]
 9   value              463962 non-null  object        
 10  valuenum           463962 non-null  float64       
 11  valueuom           464163 non-null  object        
 12  ref_range_lower    464163 non-null  float64       
 13  ref_range_upper    464163 non-null  float64 

In [26]:
# sub1_abga.to_csv('../outputs/sub1_abga.csv', index=False, encoding='utf-8-sig')

In [ ]:

# Remember to close the SSH tunnel
tunnel.close()